In [23]:
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import Dataset, DataLoader

In [6]:
""" Setting up a simple neural network model"""
class NeuralNetwork(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(NeuralNetwork, self).__init__()
        self.hidden = nn.Linear(n_inputs, n_hidden)
        self.output = nn.Linear(n_hidden, n_outputs)
        self.activation = nn.ReLU()

    def forward(self, x):
        return self.output(self.activation(self.hidden(x)))
    

In [5]:
"""Setting up a simple neural network model with dropout"""
class NeuralNetworkDropout(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs, p):
        super(NeuralNetworkDropout, self).__init__()
        self.p = p
        self.network = nn.Sequential(nn.Linear(n_inputs, n_hidden),
                                     nn.ReLU(),
                                     nn.Linear(n_hidden, n_hidden*2),
                                     nn.ReLU(),
                                     nn.Dropout(self.p),
                                     nn.Linear(n_hidden*2, n_outputs))
    
    def forward(self, x):
        return self.network(x)

In [7]:
model = NeuralNetworkDropout(3, 4, 3, 0.1)

In [25]:
x = torch.randn(100, 3)

y = x * np.sin(x) + 0.01

In [27]:
class ExampleDataset(Dataset):
    def __init__(self, x, y):
        super(ExampleDataset, self).__init__()
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.x.shape[0]

In [28]:
dataset = ExampleDataset(x, y)

In [29]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [30]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

criterion = nn.MSELoss()

epochs = 100
for epoch in range(epochs):
    for i, (x, y) in enumerate(dataloader):
        y_hat = model(x)
        loss = criterion(y_hat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
with torch.no_grad():
    model.eval()